In [1]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime, timedelta
from pandas import DataFrame, concat, date_range, ExcelWriter, to_datetime
import os
from numpy import isnan, inf, mean
import time
import json
from Calculator import Calculator

from utils import getSchema, changedType


parent = os.path.dirname(os.path.abspath("__file__"))
output_path = os.path.join(parent, "Output", "HundredBreakOut")
if not os.path.isdir(output_path):
    os.makedirs(output_path)

c:\users\kevin\appdata\local\programs\python\python37\lib\site-packages\mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


In [2]:
def StockInterDay(start_date:[str,datetime]=datetime(2021,4,14), 
                  end_date:[str,datetime]=datetime(2021,4,14),
                  tickers:[str,list]=''):
    if isinstance(start_date, datetime):
        start_date = start_date.strftime("%Y-%m-%d")
    if isinstance(end_date, datetime):
        end_date = end_date.strftime("%Y-%m-%d")
    if isinstance(tickers, str):
        tickers = tickers.split(',')
    schema = getSchema('TWSE')
    table = schema['historicalPrice']
    data = list(table.find({'Date':{'$gte':start_date, '$lte':end_date}, "Ticker" :{"$in":tickers}}))
    return data

def StockList():
    schema = getSchema('TWSE')
    table = schema['StockList']
    last_date = sorted(table.distinct("UpdateDate"))[-1]
    data = list(table.find({"UpdateDate":{"$eq":last_date}, "Industry" :{"$ne":""}}))
    return data

In [3]:
bt_list = StockList()

In [4]:
def get_commission(price:float, multiplier:int=1000, qty=1, long:bool=1, dayTrade:bool=False):
    """
    計算個別部位的單邊交易成本

    Params:
        symbol : 商品代碼
        exchange : 交易所
        cost : 交易價格
        multiplier : 價格還原現金之乘數
            例如:
                股票 : 1張 = 1,000股，10元的股票還原現金價值，即為10 *1,000 = 10,000元
                期貨 : 台指期1點200元，假設現在10,000點，則一口台股的價值為 200 * 10,000 = 2,000,000
        qty : 買賣口數或張數
        Real : 是否為實單, default = False
        direction : 交易方向 進場(買賣)或出場
            P.S. 股票交易的交易稅是出場才計算
    """
    commission = price * (0.1425 / 100) * multiplier * qty
    commission = 20 if commission < 20 else commission
    tax = price * (0.3 / 100) * multiplier * qty
    if dayTrade:
        fee /= 2
#     tradeCost = commission# * 0.6
    if not long:
        return commission, tax
    return commission, 0

In [5]:
def CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos:int = 1, qty:int = 1, multiplier:int=1000):
    pnl = (exit_price - entry_price) * pos * qty * multiplier
    entry_com, entry_tax = get_commission(entry_price, multiplier, qty, long = pos > 0)
    exit_com, exit_tax = get_commission(exit_price, multiplier, qty, long = not (pos > 0))
#     print(entry_date, exit_date, entry_price, exit_price,entry_com,entry_tax,exit_com,exit_tax)
    return {
        'EntryDate':entry_date,
        'ExitDate':exit_date,
        'EntryPrice':entry_price,
        'ExitPrice':exit_price,
        'EntryCommission':round(entry_com),
        'ExitCommission':round(exit_com),
        'EntryTax':round(entry_tax),
        'ExitTax':round(exit_tax),
        'TotalCost':round(entry_com)+round(exit_com)+round(entry_tax)+round(exit_tax),
        'HoldingPeriod':(exit_date-entry_date).days,
        'Net':round(pnl - (round(entry_com)+round(exit_com)+round(entry_tax)+round(exit_tax))),
        'Ret':round((pnl - (round(entry_com)+round(exit_com)+round(entry_tax)+round(exit_tax))) / (entry_price * 1000),4),
        'MaxPriceBetweeenHolding':max_price,
        'MaxRetBetweeenHolding':round(max_price/entry_price-1,4),
        'MinPriceBetweeenHolding':min_price,
        'MinRetBetweeenHolding':round(min_price/entry_price-1,4)
    }

In [6]:
def HundredBreakout(df:DataFrame, **kwargs):
    entry_date = None
    exit_date = None
    entry_price = 0
    exit_price = 0
    max_price = 0
    min_price = inf
    sig = pos = 0
    num_breakout_day = int(kwargs.get('num_breakout_day', 100))
    take_profit = float(kwargs.get('take_profit', .1))
    stop_loss = float(kwargs.get('stop_loss', .1))
    
    result = []
    for i, row in enumerate(df.itertuples()):
        if i < num_breakout_day: continue
        # Check Signal without pos
        if not sig and not pos:
            last_highest_close = df.iloc[-num_breakout_day-i:-i].Close.max()
            if row.Close > last_highest_close:
                sig = -1
        # Entry Market
        elif sig and not pos:
#             if row.Ticker == '1538':print(row)
            if not row.Volume or isnan(row.Open): continue
#             print(row)
            pos, sig = sig, 0
            max_price = min_price = entry_price = row.Open
            entry_date = row.Date
            max_price = max(row.High, max_price)
            min_price = min(row.Low, min_price)
        # Check Signal with pos
        elif not sig and pos:
            max_price = max(row.High, entry_price)
            min_price = min(row.Low, entry_price)
            if (row.High / entry_price - 1 >= take_profit):
                sig = -pos
#                 exit_price = row.High
            if (row.Low / entry_price - 1 <= -stop_loss):
                sig = -pos
#                 exit_price = row.Low
        # Exit Market
        elif sig and pos:
            
            if not row.Volume or isnan(row.Open): continue
#             print(row)
            exit_price = row.Open
            exit_date = row.Date
            res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
            result.append(res)
            entry_date = None
            exit_date = None
            entry_price = 0
            exit_price = 0
            max_price = 0
            min_price = inf
            sig = pos = 0
        if i == df.shape[0]-1 and pos:
            exit_price = row.Close
            exit_date = row.Date
            res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
            result.append(res)
            entry_date = None
            exit_date = None
            entry_price = 0
            exit_price = 0
            max_price = 0
            min_price = inf
            sig = pos = 0
#     print()
            
    return result
            
            

In [7]:
def Backtest(strategy:callable, ticker:str, dt:datetime=datetime.today(), bt_period = 5, params:dict={}):
    data = StockInterDay(dt+timedelta(-365*bt_period), dt, ticker)
    if not data:return []
    df = DataFrame(data)
    print(f"From {df.Date.iloc[0]} to {df.Date.iloc[-1]}")
#     print(df.iloc[0])
    df.Date = to_datetime(df.Date)
    for col in 'Open,High,Low,Close,Volume'.split(','):
        df[col] = df[col].apply(changedType)
    return strategy(df, **params)
    

In [9]:
results = {}
for ticker_info in bt_list:
    print(f"============= Backtest {ticker_info['Ticker']}=============")
    results[ticker_info['Ticker']] = Backtest(HundredBreakout, ticker_info['Ticker'], bt_period=10)
#     break

============= Backtest 1101=============
From 2012-06-18 to 2022-06-13
============= Backtest 1102=============
From 2012-06-18 to 2022-06-13
============= Backtest 1103=============
From 2012-06-18 to 2022-06-13
============= Backtest 1104=============
From 2012-06-18 to 2022-06-13
============= Backtest 1108=============
From 2012-06-18 to 2022-06-13
============= Backtest 1109=============
From 2012-06-18 to 2022-06-13
============= Backtest 1110=============
From 2012-06-18 to 2022-06-13
============= Backtest 1201=============
From 2012-06-18 to 2022-06-13
============= Backtest 1203=============
From 2012-06-18 to 2022-06-13
============= Backtest 1210=============
From 2012-06-18 to 2022-06-13
============= Backtest 1213=============
From 2012-06-18 to 2022-06-13
============= Backtest 1215=============
From 2012-06-18 to 2022-06-13
============= Backtest 1216=============
From 2012-06-18 to 2022-06-13
============= Backtest 1217=============
From 2012-06-18 to 2022-06-13
======

From 2012-06-18 to 2022-06-13
============= Backtest 1527=============
From 2012-06-18 to 2022-06-13
============= Backtest 1528=============
From 2012-06-18 to 2022-06-13
============= Backtest 1529=============
From 2012-06-18 to 2022-06-13
============= Backtest 1530=============
From 2012-06-18 to 2022-06-13
============= Backtest 1531=============
From 2012-06-18 to 2022-06-13
============= Backtest 1532=============
From 2012-06-18 to 2022-06-13
============= Backtest 1533=============
From 2012-06-18 to 2022-06-13
============= Backtest 1535=============
From 2012-06-18 to 2022-06-13
============= Backtest 1536=============
From 2012-06-18 to 2022-06-13
============= Backtest 1537=============
From 2012-06-18 to 2022-06-13
============= Backtest 1538=============
From 2012-06-18 to 2022-06-13
============= Backtest 1539=============
From 2012-06-18 to 2022-06-13
============= Backtest 1540=============
From 2012-06-18 to 2022-06-13
============= Backtest 1541=============
From 2

============= Backtest 2102=============
From 2012-06-18 to 2022-06-13
============= Backtest 2103=============
From 2012-06-18 to 2022-06-13
============= Backtest 2104=============
From 2012-06-18 to 2022-06-13
============= Backtest 2105=============
From 2012-06-18 to 2022-06-13
============= Backtest 2106=============
From 2012-06-18 to 2022-06-13
============= Backtest 2107=============
From 2012-06-18 to 2022-06-13
============= Backtest 2108=============
From 2012-06-18 to 2022-06-13
============= Backtest 2109=============
From 2012-06-18 to 2022-06-13
============= Backtest 2114=============
From 2012-06-18 to 2022-06-13
============= Backtest 2115=============
From 2013-12-25 to 2022-06-13
============= Backtest 2201=============
From 2012-06-18 to 2022-06-13
============= Backtest 2204=============
From 2012-06-18 to 2022-06-13
============= Backtest 2206=============
From 2012-06-18 to 2022-06-13
============= Backtest 2207=============
From 2012-06-18 to 2022-06-13
======

============= Backtest 2439=============
From 2012-06-18 to 2022-06-13
============= Backtest 2440=============
From 2012-06-18 to 2022-06-13
============= Backtest 2441=============
From 2012-06-18 to 2022-06-13
============= Backtest 2442=============
From 2012-06-18 to 2022-06-13
============= Backtest 2443=============
From 2012-06-18 to 2022-06-13
============= Backtest 2444=============
From 2012-06-18 to 2022-06-13
============= Backtest 2449=============
From 2012-06-18 to 2022-06-13
============= Backtest 2450=============
From 2012-06-18 to 2022-06-13
============= Backtest 2451=============
From 2012-06-18 to 2022-06-13
============= Backtest 2453=============
From 2012-06-18 to 2022-06-13
============= Backtest 2454=============
From 2012-06-18 to 2022-06-13
============= Backtest 2455=============
From 2012-06-18 to 2022-06-13
============= Backtest 2457=============
From 2012-06-18 to 2022-06-13
============= Backtest 2458=============
From 2012-06-18 to 2022-06-13
======

From 2012-06-18 to 2022-06-13
============= Backtest 2838=============
From 2012-06-18 to 2022-06-13
============= Backtest 2841=============
From 2012-06-18 to 2022-04-06
============= Backtest 2845=============
From 2012-06-18 to 2022-06-13
============= Backtest 2849=============
From 2012-06-18 to 2022-06-13
============= Backtest 2850=============
From 2012-06-18 to 2022-06-13
============= Backtest 2851=============
From 2012-06-18 to 2022-06-13
============= Backtest 2852=============
From 2012-06-18 to 2022-06-13
============= Backtest 2855=============
From 2012-06-18 to 2022-06-13
============= Backtest 2867=============
From 2012-12-18 to 2022-06-13
============= Backtest 2880=============
From 2012-06-18 to 2022-06-13
============= Backtest 2881=============
From 2012-06-18 to 2022-06-13
============= Backtest 2882=============
From 2012-06-18 to 2022-06-13
============= Backtest 2883=============
From 2012-06-18 to 2022-06-13
============= Backtest 2884=============
From 2

============= Backtest 3380=============
From 2012-06-18 to 2022-06-13
============= Backtest 3383=============
From 2012-06-18 to 2022-06-13
============= Backtest 3406=============
From 2012-06-18 to 2022-06-13
============= Backtest 3413=============
From 2015-07-28 to 2022-06-13
============= Backtest 3416=============
From 2012-06-18 to 2022-06-13
============= Backtest 3419=============
From 2012-06-18 to 2022-06-13
============= Backtest 3432=============
From 2012-06-18 to 2022-06-13
============= Backtest 3437=============
From 2014-07-09 to 2022-06-13
============= Backtest 3443=============
From 2012-06-18 to 2022-06-13
============= Backtest 3450=============
From 2012-06-18 to 2022-06-13
============= Backtest 3454=============
From 2012-06-18 to 2022-06-13
============= Backtest 3481=============
From 2012-06-18 to 2022-06-13
============= Backtest 3494=============
From 2012-06-18 to 2022-06-13
============= Backtest 3501=============
From 2012-06-18 to 2022-06-13
======

From 2012-06-18 to 2022-06-13
============= Backtest 4960=============
From 2012-06-18 to 2022-06-13
============= Backtest 4961=============
From 2012-06-18 to 2022-06-13
============= Backtest 4967=============
From 2012-06-18 to 2022-06-13
============= Backtest 4968=============
From 2015-11-13 to 2022-06-13
============= Backtest 4976=============
From 2012-11-20 to 2022-06-13
============= Backtest 4977=============
From 2013-11-20 to 2022-06-13
============= Backtest 4989=============
From 2012-06-18 to 2022-06-13
============= Backtest 4994=============
From 2012-06-18 to 2022-06-13
============= Backtest 4999=============
From 2013-06-03 to 2022-06-13
============= Backtest 5007=============
From 2012-06-18 to 2022-06-13
============= Backtest 5203=============
From 2012-06-18 to 2022-06-13
============= Backtest 5215=============
From 2012-06-18 to 2022-06-13
============= Backtest 5225=============
From 2012-11-05 to 2022-06-13
============= Backtest 5234=============
From 2

============= Backtest 6456=============
From 2015-06-12 to 2022-06-13
============= Backtest 6464=============
From 2015-12-16 to 2022-06-13
============= Backtest 6477=============
From 2016-06-21 to 2022-06-13
============= Backtest 6491=============
From 2014-12-30 to 2022-06-13
============= Backtest 6504=============
From 2013-05-07 to 2022-06-13
============= Backtest 6505=============
From 2012-06-18 to 2022-06-13
============= Backtest 6515=============
From 2021-01-20 to 2022-06-13
============= Backtest 6525=============
From 2016-04-12 to 2022-06-13
============= Backtest 6531=============
From 2016-05-31 to 2022-06-13
============= Backtest 6533=============
From 2017-03-14 to 2022-06-13
============= Backtest 6541=============
From 2015-08-13 to 2022-06-13
============= Backtest 6552=============
From 2017-01-10 to 2022-06-13
============= Backtest 6558=============
From 2015-11-17 to 2022-06-13
============= Backtest 6573=============
From 2017-09-26 to 2022-06-13
======

============= Backtest 9937=============
From 2012-06-18 to 2022-06-13
============= Backtest 9938=============
From 2012-06-18 to 2022-06-13
============= Backtest 9939=============
From 2012-06-18 to 2022-06-13
============= Backtest 9940=============
From 2012-06-18 to 2022-06-13
============= Backtest 9941=============
From 2012-06-18 to 2022-06-13
============= Backtest 9942=============
From 2012-06-18 to 2022-06-13
============= Backtest 9943=============
From 2012-06-18 to 2022-06-13
============= Backtest 9944=============
From 2012-06-18 to 2022-06-13
============= Backtest 9945=============
From 2012-06-18 to 2022-06-13
============= Backtest 9946=============
From 2012-06-18 to 2022-06-13
============= Backtest 9955=============
From 2012-06-18 to 2022-06-13
============= Backtest 9958=============
From 2012-06-18 to 2022-06-13
============= Backtest 1312A=============
From 2012-06-18 to 2022-06-13
============= Backtest 2002A=============
From 2012-06-18 to 2022-06-13
====

ValueError: cannot convert float NaN to integer

# Summary

In [10]:
summary_ = []
bt_day = datetime.today()
bt_day_str = bt_day.strftime("%Y%m%d")
for ticker, result in results.items():
    total_cost = 0
#     total_pnl = 0
    total_net = 0
    trade_num = len(result)
    win_num = 0
    loss_num = 0
    hold_period = []
    for res in result:
        total_net += res['Net']
        total_cost += res['TotalCost']
        win_num += int(res['Net'] > 0)
        loss_num += int(res['Net'] < 0)
        hold_period.append(res['HoldingPeriod'])
    summary_.append({
        '代號':ticker,
        '總成本':total_cost,
        '總損益(淨)':total_net,
        '獲利次數':win_num,
        '損失次數':loss_num,
        '總交易次數':trade_num,
        '勝率%':round(win_num / trade_num*100,2) if trade_num else 0,
        '平均持倉時間(日)':mean(hold_period)
    })
#     if result:
#         DataFrame(result).to_csv(os.path.join(output_path, f'{ticker}_{bt_day_str}.csv'), index=False)
        

In [11]:
sum_df = DataFrame(summary_)#.sort_values(['勝率%','總交易次數'], ascending=False)

In [12]:
sum_df['總損益(淨)'].sum()

-9804853

In [201]:
sum_df.to_csv(os.path.join(output_path, 'Summary.csv'),index=False, encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'F:\\SiteProject\\StrategyDev\\台股波段\\無腦交易\\Output\\HundredBreakOut\\Summary.csv'

In [13]:
sum_df_prob_sorted = sum_df[sum_df['勝率%'] > 60] # sum_df.sort_values(['勝率%'], ascending=False)
sum_df_trade_num_sorted = sum_df[sum_df.總交易次數 >= 20]#sum_df.sort_values(['總交易次數'], ascending=False)
sum_df_holding_period_sorted = sum_df[sum_df['平均持倉時間(日)'] <= 40]

In [14]:
top_num = 100
prob_Ticker = sum_df_prob_sorted.代號 # .iloc[:top_num]
trade_num_Ticker = sum_df_trade_num_sorted.代號 # .iloc[:top_num]
holding_period_Ticker = sum_df_holding_period_sorted.代號
suitable_Ticker = list(set(prob_Ticker).intersection(trade_num_Ticker).intersection(holding_period_Ticker))

In [15]:
print(sum_df[sum_df.代號.isin(suitable_Ticker)].shape)
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False) # ['總損益(淨)'].sum()

(62, 8)


,代號,總成本,總損益(淨),獲利次數,損失次數,總交易次數,勝率%,平均持倉時間(日)
192,1809,2618,16132,22,8,30,73.33,37.066667
999,2724,9236,60964,31,12,43,72.09,29.395349
677,4807,7454,75746,15,6,21,71.43,36.333333
535,3041,4608,38442,25,10,35,71.43,25.857143
184,1783,6712,34438,24,10,34,70.59,20.088235
...,...,...,...,...,...,...,...,...
632,3708,15699,55901,14,9,23,60.87,21.695652
575,3346,9267,45633,17,11,28,60.71,25.321429
49,1340,15120,46630,20,13,33,60.61,37.848485
638,4108,7957,25943,23,15,38,60.53,36.657895
